## EDA and Cleaning - SourceStack datasets

This notebook focuses on exploration and cleaning of two datasets I obtained by calling SourceStack API\
The first dataset comes from: **June 9, 2023**\
and the more recent one from: **April 2, 2024**

### Initial Exploration
1. shape
2. dtypes
3. missing values


### Cleaning
1. parsing strings containing datetimes to dates
2. converting strings containing a list to list of strings
3. convertsing numerical data from strings to Int/Float
5. identify dirty categories

#### Let's read in the data and have a look at its shape, columns and values

In [2]:
!pip install "polars_ds[plot]"

In [3]:
import sys

In [4]:
print(sys.executable)

/home/anopsy/Portfolio/sourcestack/sstack/bin/python


In [5]:
import polars as pl

In [6]:
old_data_path = "/home/anopsy/Portfolio/sourcestack/data/9june2023.csv"
new_data_path = "/home/anopsy/Portfolio/sourcestack/data/2april2024.csv"

In [7]:
old_df = pl.read_csv(old_data_path, try_parse_dates=False)
new_df = pl.read_csv(new_data_path, try_parse_dates=False)

In [8]:
print(f"Shape of the old data1 is:{old_df.shape}")
print(f"Shape of the new data is:{new_df.shape}")

Shape of the old data1 is:(50000, 16)
Shape of the new data is:(50000, 16)


In [9]:
old_df.head()

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str
"""Backend Developer""","""Praha, Czech Republic""",null,null,"""IBM""",null,"""[Docker, GraphQL, NoSQL, IBM, …","""[Container Orchestration, Quer…","""[Software]""",null,"""""","""pl""","""Praha""","""Czech Republic""","""2023-03-13 05:12:29""","""2023-06-05 13:43:49"""
"""Manufacturing Engineer""",null,"""Full-Time""",false,null,null,"""[Sigma]""","""[Tools, Serverless]""","""[Manufacturing]""",null,"""""","""en""","""Sterling Heights""","""United States""","""2021-10-09 00:00:00""","""2023-05-24 05:35:57"""
"""Design Engineer, Motorized Pro…","""520 S Byrkit St Mishawaka, Ind…","""Full-Time""",null,"""ABI Attachments""","""Bachelors""","""[]""","""[]""","""[Design]""","""Senior IC""","""""","""en""","""Mishawaka""","""United States""","""2023-04-28 03:04:28""","""2023-05-19 14:48:10"""
"""Cybersecurity Engineer""",null,"""Full-Time""",false,null,"""Bachelors""","""[AWS, Qualys, Splunk]""","""[Compute, Logging & Monitoring…","""[Cybersecurity, Security]""",null,"""""","""en""","""Herndon""","""United States""","""2023-04-03 00:00:00""","""2023-05-28 11:47:09"""
"""Your Career so choose wisely w…","""Kolkata, India""","""Full-Time""",false,"""Adeeba e Services""",null,"""[Objective-C, Subversion, Swif…","""[Cloud Native Storage, Program…","""[Software]""",null,"""""","""en""","""Kolkata""","""India""","""2017-01-17 11:35:48""","""2023-05-30 11:51:08"""


In [10]:
new_df.head()

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str
"""Dir, Engineering NPD, Critical…","""Dominican Republic-Nave 25-Mer…","""Full-Time""",null,"""DR""","""Bachelors""","""[Microsoft]""","""[]""","""[]""",null,null,"""en""",null,"""Dominican Republic""","""2024-03-04 00:00:00""","""2024-03-26 08:03:11"""
"""Software Engineer - Embedded""","""Dresden or Hartmannsdorf, Sach…","""Full-Time""",null,"""Manning Global""",null,"""[Linux]""","""[OS]""","""[Software, IT]""",null,null,"""en""","""Dresden or Hartmannsdorf""","""Germany""","""2024-02-15 00:00:00""","""2024-04-01 09:40:27"""
"""Embedded Software Test Enginee…","""Brisbane, CA""","""Full-Time""",null,"""Avive""",null,"""[Linux, C++]""","""[OS, Programming Languages, OS…","""[Software]""",null,"""150000.00""","""en""","""Brisbane""","""Australia""","""2023-10-23 00:00:00""","""2024-04-01 15:25:43"""
"""Manufacturing Engineering Mana…","""Monroe, WI""","""Full-Time""",null,"""United Future""",null,"""[]""","""[]""","""[Manufacturing]""","""Manager""","""1.00""","""en""","""Monroe""","""United States""","""2024-03-27 20:18:23""","""2024-03-28 20:24:27"""
"""Vom Lager zum Wächter | Direkt…","""Ennepetal, Nordrhein-Westfalen…","""Full-Time""",null,"""RUHR VERMITTLUNG""",null,"""[WhatsApp, Vercel]""","""[Communications, VoIP, Serverl…","""[Security]""",null,null,"""de""","""Dortmund""","""Germany""","""2024-03-27 12:31:19""","""2024-03-31 11:16:19"""


### Initial explorations of unprocessed dataframes

#### Shape
Both datasets contain **50000 records** \
each record is represented by **16 features**

#### Dtypes
15 of the features are currently String - datatype\
1 feature is Bool

#### Missing values
The datasets contain **null values** and **empty strings**

In [11]:
pl.Config.set_tbl_width_chars(
    200
)  # setting wide format but it doesn't work that well for jupyter notebook

polars.config.Config

Let's have a look at the sample method, so I can have a look at some more records and remember that I can use .sample in the future.

In [12]:
old_df.sample(3)

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str
"""Senior Project Engineer""","""New South Wales""","""Full-Time""",null,"""Seymour Whyte""","""Bachelors""","""[]""","""[]""","""[]""","""Senior IC""","""""","""en-au""",null,"""Wales""","""2023-06-05 00:00:00""","""2023-06-05 16:19:13"""
"""Sr Associate, Vendor Security …","""Mexico/Remote""","""Full-Time""",true,"""KTSA""",null,"""[Excel, Express]""","""[Spreadsheets, Midsize Custome…","""[Security, Management Consulti…","""Senior IC""","""""","""en""","""Mexico/Remote""","""Mexico""","""2023-01-25 00:00:00""","""2023-05-20 04:52:38"""
"""Site Reliability Engineer""","""Tehran, 23, 0098 Iran""","""Full-Time""",null,"""Snapp""",null,"""[Facebook, Grafana, Kubernetes…","""[Orchestration & Management, D…","""[]""",null,null,"""en""","""Tehran""","""Iran""","""2023-02-12 00:00:00""","""2023-06-09 04:11:40"""


In [13]:
new_df.sample(3)

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str
"""Engineer - Mechanical Componen…","""IN-TN-COIMBATORE-CHETTIPALAYAM…","""Full-Time""",null,"""GE Oil & Gas India""","""Bachelors""","""[Excel]""","""[Treasury, Misc Biz Tools, Ent…","""[India-Related]""",null,null,"""en""",null,"""India""","""2024-03-20 00:00:00""","""2024-03-25 12:16:08"""
"""Global Program Manager, Direct…","""United States - California - S…","""Full-Time""",null,"""1S2 Seagen""","""Bachelors""","""[GPT-3]""","""[Automated Content Generation]""","""[]""","""Exec""","""215500.00""","""en""",null,"""United States""","""2024-03-07 00:00:00""","""2024-03-25 18:13:00"""
"""DevOps Engineer-AR-Buenos Aire…","""La Plata, Buenos Aires, Argent…",null,null,"""sistemasglT1""",null,"""[Chef, Linux, VirtualBox, Link…","""[Compute, IaaS, OSS, Provision…","""[DevOps, Marketing]""",null,null,"""en-us""","""La Plata""","""Argentina""","""2024-03-21 02:00:00""","""2024-03-23 14:49:46"""


#### Add column that will help us identify if the record comes from 2023 or 2024 and concatenate both dataframes into one

In [14]:
# adding static columns with a string helping identify the df
old_df = old_df.with_columns(pl.lit("no").alias("new"))
new_df = new_df.with_columns(pl.lit("yes").alias("new"))

In [15]:
old_df.head()

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,new
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Backend Developer""","""Praha, Czech Republic""",null,null,"""IBM""",null,"""[Docker, GraphQL, NoSQL, IBM, …","""[Container Orchestration, Quer…","""[Software]""",null,"""""","""pl""","""Praha""","""Czech Republic""","""2023-03-13 05:12:29""","""2023-06-05 13:43:49""","""no"""
"""Manufacturing Engineer""",null,"""Full-Time""",false,null,null,"""[Sigma]""","""[Tools, Serverless]""","""[Manufacturing]""",null,"""""","""en""","""Sterling Heights""","""United States""","""2021-10-09 00:00:00""","""2023-05-24 05:35:57""","""no"""
"""Design Engineer, Motorized Pro…","""520 S Byrkit St Mishawaka, Ind…","""Full-Time""",null,"""ABI Attachments""","""Bachelors""","""[]""","""[]""","""[Design]""","""Senior IC""","""""","""en""","""Mishawaka""","""United States""","""2023-04-28 03:04:28""","""2023-05-19 14:48:10""","""no"""
"""Cybersecurity Engineer""",null,"""Full-Time""",false,null,"""Bachelors""","""[AWS, Qualys, Splunk]""","""[Compute, Logging & Monitoring…","""[Cybersecurity, Security]""",null,"""""","""en""","""Herndon""","""United States""","""2023-04-03 00:00:00""","""2023-05-28 11:47:09""","""no"""
"""Your Career so choose wisely w…","""Kolkata, India""","""Full-Time""",false,"""Adeeba e Services""",null,"""[Objective-C, Subversion, Swif…","""[Cloud Native Storage, Program…","""[Software]""",null,"""""","""en""","""Kolkata""","""India""","""2017-01-17 11:35:48""","""2023-05-30 11:51:08""","""no"""


In [16]:
# concatenating old and new data
whole_df = old_df.vstack(new_df)

print(whole_df.shape)

(100000, 17)


### Cleaning

#### 1. Converting 'job_published_at', 'last_indexed' to Date

In [17]:
whole_df = whole_df.with_columns(
    pl.col("job_published_at", "last_indexed").str.to_datetime().cast(pl.Date)
)

In [18]:
whole_df.head()

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,new
str,str,str,bool,str,str,str,str,str,str,str,str,str,str,date,date,str
"""Backend Developer""","""Praha, Czech Republic""",null,null,"""IBM""",null,"""[Docker, GraphQL, NoSQL, IBM, …","""[Container Orchestration, Quer…","""[Software]""",null,"""""","""pl""","""Praha""","""Czech Republic""",2023-03-13,2023-06-05,"""no"""
"""Manufacturing Engineer""",null,"""Full-Time""",false,null,null,"""[Sigma]""","""[Tools, Serverless]""","""[Manufacturing]""",null,"""""","""en""","""Sterling Heights""","""United States""",2021-10-09,2023-05-24,"""no"""
"""Design Engineer, Motorized Pro…","""520 S Byrkit St Mishawaka, Ind…","""Full-Time""",null,"""ABI Attachments""","""Bachelors""","""[]""","""[]""","""[Design]""","""Senior IC""","""""","""en""","""Mishawaka""","""United States""",2023-04-28,2023-05-19,"""no"""
"""Cybersecurity Engineer""",null,"""Full-Time""",false,null,"""Bachelors""","""[AWS, Qualys, Splunk]""","""[Compute, Logging & Monitoring…","""[Cybersecurity, Security]""",null,"""""","""en""","""Herndon""","""United States""",2023-04-03,2023-05-28,"""no"""
"""Your Career so choose wisely w…","""Kolkata, India""","""Full-Time""",false,"""Adeeba e Services""",null,"""[Objective-C, Subversion, Swif…","""[Cloud Native Storage, Program…","""[Software]""",null,"""""","""en""","""Kolkata""","""India""",2017-01-17,2023-05-30,"""no"""


#### 2. Converting 'tags_matched', 'tag_categories', 'categories' from str to list[str]

In [19]:
def string_to_nested(df, cols):
    """
    takes a df and list of columns that contain strings with lists
    and turns them into nested datatype List
    """
    for col in cols:
        df = df.with_columns(
            pl.col(col).str.extract_all(r"\w+").cast(pl.List(pl.String))
        )
    return df

In [20]:
cols_to_change = ["tags_matched", "tag_categories", "categories"]
whole_df = string_to_nested(whole_df, cols_to_change)

In [21]:
whole_df.head()

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,new
str,str,str,bool,str,str,list[str],list[str],list[str],str,str,str,str,str,date,date,str
"""Backend Developer""","""Praha, Czech Republic""",null,null,"""IBM""",null,"[""Docker"", ""GraphQL"", … ""Cloud""]","[""Container"", ""Orchestration"", … ""Databases""]","[""Software""]",null,"""""","""pl""","""Praha""","""Czech Republic""",2023-03-13,2023-06-05,"""no"""
"""Manufacturing Engineer""",null,"""Full-Time""",false,null,null,"[""Sigma""]","[""Tools"", ""Serverless""]","[""Manufacturing""]",null,"""""","""en""","""Sterling Heights""","""United States""",2021-10-09,2023-05-24,"""no"""
"""Design Engineer, Motorized Pro…","""520 S Byrkit St Mishawaka, Ind…","""Full-Time""",null,"""ABI Attachments""","""Bachelors""",[],[],"[""Design""]","""Senior IC""","""""","""en""","""Mishawaka""","""United States""",2023-04-28,2023-05-19,"""no"""
"""Cybersecurity Engineer""",null,"""Full-Time""",false,null,"""Bachelors""","[""AWS"", ""Qualys"", ""Splunk""]","[""Compute"", ""Logging"", … ""Security""]","[""Cybersecurity"", ""Security""]",null,"""""","""en""","""Herndon""","""United States""",2023-04-03,2023-05-28,"""no"""
"""Your Career so choose wisely w…","""Kolkata, India""","""Full-Time""",false,"""Adeeba e Services""",null,"[""Objective"", ""C"", … ""Git""]","[""Cloud"", ""Native"", … ""Control""]","[""Software""]",null,"""""","""en""","""Kolkata""","""India""",2017-01-17,2023-05-30,"""no"""


#### 3. Converting 'comp_est' from str to int

In [22]:
whole_df = whole_df.with_columns(
    pl.col("comp_est").cast(pl.Float64, strict=False).alias("compensation")
)
# polars can handle str->float->int
# casting didn't work for Int64 but it did for Float with strict=False, strict=False turned empty strings to nulls
# it works after all I think the problem was I tried to cast t oint32 and because of huge numbers it didn't work
# now it works with Int64

In [23]:
whole_df.head()

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,new,compensation
str,str,str,bool,str,str,list[str],list[str],list[str],str,str,str,str,str,date,date,str,f64
"""Backend Developer""","""Praha, Czech Republic""",null,null,"""IBM""",null,"[""Docker"", ""GraphQL"", … ""Cloud""]","[""Container"", ""Orchestration"", … ""Databases""]","[""Software""]",null,"""""","""pl""","""Praha""","""Czech Republic""",2023-03-13,2023-06-05,"""no""",null
"""Manufacturing Engineer""",null,"""Full-Time""",false,null,null,"[""Sigma""]","[""Tools"", ""Serverless""]","[""Manufacturing""]",null,"""""","""en""","""Sterling Heights""","""United States""",2021-10-09,2023-05-24,"""no""",null
"""Design Engineer, Motorized Pro…","""520 S Byrkit St Mishawaka, Ind…","""Full-Time""",null,"""ABI Attachments""","""Bachelors""",[],[],"[""Design""]","""Senior IC""","""""","""en""","""Mishawaka""","""United States""",2023-04-28,2023-05-19,"""no""",null
"""Cybersecurity Engineer""",null,"""Full-Time""",false,null,"""Bachelors""","[""AWS"", ""Qualys"", ""Splunk""]","[""Compute"", ""Logging"", … ""Security""]","[""Cybersecurity"", ""Security""]",null,"""""","""en""","""Herndon""","""United States""",2023-04-03,2023-05-28,"""no""",null
"""Your Career so choose wisely w…","""Kolkata, India""","""Full-Time""",false,"""Adeeba e Services""",null,"[""Objective"", ""C"", … ""Git""]","[""Cloud"", ""Native"", … ""Control""]","[""Software""]",null,"""""","""en""","""Kolkata""","""India""",2017-01-17,2023-05-30,"""no""",null


In [24]:
whole_df.filter(pl.col("compensation") > 0).shape

(14962, 18)

In [25]:
whole_df.select(pl.col("comp_est").value_counts(sort=True)).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_4025,column_4026,column_4027,column_4028,column_4029,column_4030,column_4031,column_4032,column_4033,column_4034,column_4035,column_4036,column_4037,column_4038,column_4039,column_4040,column_4041,column_4042,column_4043,column_4044,column_4045,column_4046,column_4047,column_4048,column_4049,column_4050,column_4051,column_4052,column_4053,column_4054,column_4055,column_4056,column_4057,column_4058,column_4059,column_4060,column_4061
struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],…,struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2]
"{null,47737}","{"""",36939}","{""None"",353}","{""1.00"",209}","{""150000.00"",152}","{""6.00"",143}","{""100000.00"",123}","{""120000.00"",122}","{""40000.00"",120}","{""4.00"",111}","{""110000.00"",107}","{""60000.00"",100}","{""140000.00"",100}","{""36000.00"",99}","{""34000.00"",98}","{""160000.00"",93}","{""2.00"",93}","{""80000.00"",92}","{""17.00"",92}","{""130000.00"",88}","{""50000.00"",87}","{""90000.00"",87}","{""125000.00"",86}","{""33.00"",84}","{""7.00"",84}","{""190000.00"",82}","{""15.00"",82}","{""30000.00"",81}","{""135000.00"",81}","{""16.00"",80}","{""105000.00"",78}","{""500.00"",78}","{""115000.00"",77}","{""3.00"",76}","{""5.00"",76}","{""145000.00"",75}","{""1000.00"",74}",…,"{""2620.8"",1}","{""23220.00"",1}","{""16.4"",1}","{""6000000.00"",1}","{""74561.27"",1}","{""5997.00"",1}","{""127450.00"",1}","{""87570.00"",1}","{""350000.0"",1}","{""4650.00"",1}","{""49.0"",1}","{""3550.0"",1}","{""122000.0"",1}","{""106114.00"",1}","{""237750.00"",1}","{""70.0"",1}","{""83525.00"",1}","{""20.95"",1}","{""130000150000.00"",1}","{""79770.76000000001"",1}","{""123550.00"",1}","{""320000.00"",1}","{""19800000.00"",1}","{""119500.0"",1}","{""173375.00"",1}","{""28800.00"",1}","{""76323.00"",1}","{""84180.00"",1}","{""88875.00"",1}","{""285000.00"",1}","{""196300.00"",1}","{""19.395"",1}","{""130.0"",1}","{""20.305"",1}","{""49785.00"",1}","{""52880.00"",1}","{""89838.00"",1}"


#### Duplicates

In [26]:
no_dupl = whole_df.unique()
no_dupl  # there were 333 duplicates

job_name,job_location,hours,remote,company_name,education,tags_matched,tag_categories,categories,seniority,comp_est,language,city,country,job_published_at,last_indexed,new,compensation
str,str,str,bool,str,str,list[str],list[str],list[str],str,str,str,str,str,date,date,str,f64
"""Java Senior Engineer""","""Buffalo Grove, Illinois, Unite…",null,null,"""Onbe""","""Bachelors""","[""Oracle"", ""PostgreSQL"", … ""AWS""]","[""Timeseries"", ""Database"", … ""Orchestration""]","[""Payments"", ""Performing"", ""Arts""]","""Senior IC""",null,"""en""","""Buffalo Grove""","""United States""",2024-03-12,2024-04-01,"""yes""",null
"""Principal Cyber Security Risk …",null,"""Full-Time""",null,"""EGM Executive Search and Recru…",null,[],[],"[""Recruiting"", ""Staffing"", … ""Security""]","""Staff IC""","""""","""en""",null,null,2023-04-20,2023-05-26,"""no""",null
"""Mobile full stack developer_CB…","""Pune""","""Contract""",false,"""Coders Brain Technology""",null,"[""Java"", ""Android"", … ""js""]","[""JavaScript"", ""UI"", … ""OS""]","[""Software""]",null,null,"""en""","""Pune""","""India""",2022-12-24,2024-03-21,"""yes""",null
"""Product Manager Release Test""","""Bengaluru""","""Full-Time""",false,"""FullThrottle Labs testcdsc""",null,[],[],"[""Marketing""]","""IC""","""1000000.0""","""en""","""Bengaluru""","""India""",2022-09-14,2024-03-27,"""yes""",1e6
"""Sr Firmware engineer""","""IN - Karnataka - Bangalore""","""Full-Time""",null,"""Schneider Electric""",null,"[""Schneider"", ""Lua"", … ""JavaScript""]","[""Industrial"", ""OSS"", … ""OS""]",[],"""Senior IC""","""""","""en""","""Bangalore""","""India""",2023-05-04,2023-05-23,"""no""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Senior Systems Engineer""","""490 ANNAPOLIS JUNCTION MD""","""Full-Time""",null,"""25 LGS Innovations""",null,[],[],[],"""Senior IC""","""""","""en""","""Annapolis Junction""","""United States""",2023-05-17,2023-05-19,"""no""",null
"""System Administrator with Secu…","""boston, massachusetts""","""Full-Time""",null,"""Explore Jobs Search""","""Bachelors""","[""Microsoft"", ""Red"", … ""Directory""]","[""OSS"", ""Password"", … ""Management""]","[""Security""]","""Unclear Seniority""","""""","""en""","""Boston""","""United States""",2022-06-09,2023-05-19,"""no""",null
"""Sr. Data Engineer Azure Databr…","""Pune, India""","""Contract""",null,"""Fusemachines""","""Bachelors""","[""Azure"", ""Python"", … ""Storage""]","[""Data"", ""Warehouses"", … ""CI""]","[""AI"", ""Education""]","""Senior IC""",null,"""en""","""Pune""","""India""",null,2024-03-23,"""yes""",null


In [27]:
!pip install hvplot
!pip install pyarrow

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [64]:
seniority_groups = no_dupl.group_by("seniority", "new").agg(
    pl.col("seniority").count().alias("count")
)

In [68]:
seniority_groups.hvplot.bar(
    x="seniority", y="count", color="new", subplots=True, rot=90
)
# , color="new", subplots=True

:Bars   [seniority]   (count,new)

In [88]:
from datetime import datetime

In [89]:
clean_timeline = no_dupl.filter(
    pl.col("job_published_at").is_between(
        datetime(2022, 12, 31), datetime(2024, 12, 31)
    ),
)

In [90]:
timeline = clean_timeline.group_by("job_published_at", "new").agg(
    pl.col("job_published_at").count().alias("job_count")
)
timeline

job_published_at,new,job_count
date,str,u32
2023-10-31,"""yes""",65
2023-05-19,"""yes""",19
2023-02-17,"""yes""",10
2023-03-21,"""yes""",17
2023-10-15,"""yes""",2
…,…,…
2023-09-12,"""yes""",43
2023-06-05,"""yes""",22
2023-03-20,"""yes""",12


In [92]:
pivot_timeline = timeline.pivot(
    index="job_published_at", columns="new", values="job_count"
)

In [93]:
pivot_timeline.hvplot.line(x="job_published_at", y=["yes", "no"])

:NdOverlay   [Variable]
   :Curve   [job_published_at]   (value)

In [ ]:
timeline.hvplot.line(x="job_published_at", y="job_count", color="new")

#### 4. Identify dirty categories


In [31]:
whole_df = whole_df.with_columns(pl.col("job_name").str.to_lowercase())

In [32]:
whole_df.select(pl.col("job_name").value_counts(sort=True)).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_65696,column_65697,column_65698,column_65699,column_65700,column_65701,column_65702,column_65703,column_65704,column_65705,column_65706,column_65707,column_65708,column_65709,column_65710,column_65711,column_65712,column_65713,column_65714,column_65715,column_65716,column_65717,column_65718,column_65719,column_65720,column_65721,column_65722,column_65723,column_65724,column_65725,column_65726,column_65727,column_65728,column_65729,column_65730,column_65731,column_65732
struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],…,struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2]
"{""software engineer"",814}","{""senior software engineer"",615}","{""product manager"",462}","{""devops engineer"",398}","{""data engineer"",378}","{""project engineer"",373}","{""security officer"",361}","{""electrical engineer"",354}","{""program manager"",342}","{""data analyst"",327}","{""mechanical engineer"",277}","{""full stack developer"",276}","{""software developer"",274}","{""data scientist"",247}","{""systems engineer"",236}","{""quality engineer"",235}","{""network engineer"",235}","{""security guard"",232}","{""retail front end supervisor"",202}","{""process engineer"",196}","{""manufacturing engineer"",194}","{""senior data engineer"",194}","{""engineering manager"",192}","{""application developer: cloud fullstack"",184}","{""senior devops engineer"",182}","{""sales engineer"",164}","{""senior product manager"",161}","{""senior software developer"",158}","{""technical writer"",155}","{""field service engineer"",153}","{""site reliability engineer"",147}","{""product owner"",146}","{""backend developer"",144}","{""android developer"",144}","{""ios developer"",140}","{""civil engineer"",131}","{""qa engineer"",128}",…,"{""wind engineer (offshore) - expression of interest"",1}","{""building electrical engineering intern - summer 2024"",1}","{""principal research engineer, dynamically reconfigurable real-time systems"",1}","{""software engineer (backend nodejs) - flutter studios"",1}","{""environmental engineer, scientist, or geologist"",1}","{""security field supervisor armed - county"",1}","{""engineer apprentice"",1}","{""cloud analyst"",1}","{""security attendant (seasonal)"",1}","{""junior développeur front/back-end / nodejs – fintech"",1}","{""2024 summer undergraduate intern/co-op - manufacturing engineer"",1}","{""automation qa engineer (backup)"",1}","{""embedded c software engineer with classic autosar for adas integration platform, engineering center, sibiu"",1}","{""switchgear quotations engineer"",1}","{""swqa automation and tools development engineer"",1}","{""data engineer (questionnaire)"",1}","{""werkstudent (m/w/d) für software-tests"",1}","{""chemical process engineer lead"",1}","{""data scientist, avp - people analytics"",1}","{""requirements & systems engineer"",1}","{""test automation engineer (234406)"",1}","{""designer - design studios"",1}","{""mobile developer ios/android"",1}","{""oracle integration cloud developer�// remote mexico""

In [33]:
job_names = (
    whole_df.group_by("job_name")
    .agg(pl.col("job_name").count().alias("count"))
    .sort("count", descending=True)
)

In [34]:
job_pop = job_names.filter(pl.col("count") > 50)

In [35]:
job_pop.transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99,column_100
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""software engineer""","""senior software engineer""","""product manager""","""devops engineer""","""data engineer""","""project engineer""","""security officer""","""electrical engineer""","""program manager""","""data analyst""","""mechanical engineer""","""full stack developer""","""software developer""","""data scientist""","""systems engineer""","""quality engineer""","""network engineer""","""security guard""","""retail front end supervisor""","""process engineer""","""manufacturing engineer""","""senior data engineer""","""engineering manager""","""application developer: cloud f…","""senior devops engineer""","""sales engineer""","""senior product manager""","""senior software developer""","""technical writer""","""field service engineer""","""site reliability engineer""","""product owner""","""backend developer""","""android developer""","""ios developer""","""civil engineer""","""engineer""",…,"""controls engineer""","""engineering technician""","""quality assurance engineer""","""package consultant: sap cloud …","""software engineer ii""","""senior engineer""","""machine learning engineer""","""chief engineer""","""application engineer""","""cloud engineer""","""staff software engineer""","""senior full stack developer""","""system engineer""","""embedded software engineer""","""software de recrutamento e sel…","""software architect""","""senior structural engineer""","""senior program manager""","""product engineer""","""security engineer""","""qa automation engineer""","""industrial engineer""","""service engineer""","""engineer ii""","""application developer: azure c…","""production engineer""","""senior full stack engineer""","""engineering intern""","""unarmed security officer""","""senior backend developer""","""software development engineer""","""lead engineer""","""big data engineer""","""solutions engineer""","""software engineer iii""","""technical product manager""","""electrical design engineer"""
"""814""","""615""","""462""","""398""","""378""","""373""","""361""","""354""","""342""","""327""","""277""","""276""","""274""","""247""","""236""","""235""","""235""","""232""","""202""","""196""","""194""","""194""","""192""","""184""","""182""","""164""","""161""","""158""","""155""","""153""","""147""","""146""","""144""","""144""","""140""","""131""","""128""",…,"""76""","""76""","""76""","""76""","""75""","""74""","""73""","""70""","""68""","""68""","""67""","""67""","""66""","""65""","""64""","""64""","""62""","""62""","""62""","""61""","""60""","""59""","""58""","""58""","""57""","""56""","""56""","""56""","""55""","""55""","""53""","""52""","""52""","""52""","""51""","""51""","""51"""


In [36]:
choices = job_pop.select(pl.col("job_name"))
choices.dtypes

[String]

In [37]:
choices

job_name
str
"""software engineer"""
"""senior software engineer"""
"""product manager"""
"""devops engineer"""
"""data engineer"""
…
"""big data engineer"""
"""solutions engineer"""
"""software engineer iii"""


In [38]:
whole_df.select(pl.col("company_name").value_counts(sort=True))

company_name
struct[2]
"{null,6266}"
"{""IBM"",2683}"
"{""Allied Universal"",1057}"
"{""CLBPTS"",668}"
"{""Bosch Group"",533}"
…
"{""Ground Recruitment"",1}"
"{""ramblr.ai"",1}"
"{""UPL-"",1}"


In [39]:
(
    whole_df.group_by("company_name")
    .agg(pl.col("company_name").count().alias("count"))
    .filter(pl.col("count") > 1)
    .sort("count", descending=True)
)

company_name,count
str,u32
"""IBM""",2683
"""Allied Universal""",1057
"""CLBPTS""",668
"""Bosch Group""",533
"""Schneider Electric""",397
…,…
"""Zip Co Limited""",2
"""Axians hamcos""",2
"""Cybellum""",2


Let's create a list of most common job_names, and then let's fuzzy match them with the rest

In [40]:
job_names = (
    whole_df.group_by("job_name")
    .agg(pl.col("job_name").count().alias("count"))
    .sort("count", descending=True)
)

In [41]:
job_pop = job_names.filter(pl.col("count") > 10)

In [42]:
job_pop.transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_465,column_466,column_467,column_468,column_469,column_470,column_471,column_472,column_473,column_474,column_475,column_476,column_477,column_478,column_479,column_480,column_481,column_482,column_483,column_484,column_485,column_486,column_487,column_488,column_489,column_490,column_491,column_492,column_493,column_494,column_495,column_496,column_497,column_498,column_499,column_500,column_501
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""software engineer""","""senior software engineer""","""product manager""","""devops engineer""","""data engineer""","""project engineer""","""security officer""","""electrical engineer""","""program manager""","""data analyst""","""mechanical engineer""","""full stack developer""","""software developer""","""data scientist""","""systems engineer""","""quality engineer""","""network engineer""","""security guard""","""retail front end supervisor""","""process engineer""","""senior data engineer""","""manufacturing engineer""","""engineering manager""","""application developer: cloud f…","""senior devops engineer""","""sales engineer""","""senior product manager""","""senior software developer""","""technical writer""","""field service engineer""","""site reliability engineer""","""product owner""","""android developer""","""backend developer""","""ios developer""","""civil engineer""","""qa engineer""",…,"""broadcast engineer""","""lead qa engineer""",""".net full stack developer""","""data analyst (remote)""","""marketing data analyst""","""manufacturing engineering mana…","""sr. data scientist""","""senior cloud security engineer""","""electrical/controls/automation…","""software quality assurance eng…","""sr. software developer""","""fire engineer""","""package consultant: oracle clo…","""cloud data engineer""","""project engineering manager""","""site reliability engineer (sre…","""senior software engineer - jav…","""functional safety engineer""","""senior cybersecurity engineer""","""cloud infrastructure engineer""","""engineering internship""","""data scientist ii""","""engineering specialist""","""systems integration engineer""","""principal mechanical engineer""","""lead software developer""","""senior security analyst""","""cad engineer""","""technical engineer""","""devops engineer - remote, full…","""site reliability engineer iii""","""transportation project enginee…","""office engineer""","""robotics engineer""","""field sales engineer""","""lead site reliability engineer""","""application developer: ibm clo…"
"""814""","""615""","""462""","""398""","""378""","""373""","""361""","""354""","""342""","""327""","""277""","""276""","""274""","""247""","""236""","""235""","""235""","""232""","""202""","""196""","""194""","""194""","""192""","""184""","""182""","""164""","""161""","""158""","""155""","""153""","""147""","""146""","""144""","""144""","""140""","""131""","""128""",…,"""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11""","""11"""


In [43]:
data_job = (
    whole_df.filter(pl.col("job_name").str.contains("(?i)data"))
    .group_by("job_name")
    .agg(pl.col("job_name").count().alias("count"))
    .sort("count", descending=True)
)

In [44]:
junior_job = (
    whole_df.filter(pl.col("job_name").str.contains("(?i)junior"))
    .group_by("job_name")
    .agg(pl.col("job_name").count().alias("count"))
    .sort("count", descending=True)
)

In [45]:
junior_job

job_name,count
str,u32
"""junior software engineer""",34
"""junior data scientist - dubai,…",21
"""junior data analyst""",20
"""junior software developer""",14
"""junior mechanical engineer""",8
…,…
"""junior customer support engine…",1
"""advogado junior - contencioso …",1
"""(junior) product manager medic…",1


In [46]:
intern = (
    whole_df.filter(pl.col("job_name").str.contains("(?i)intern"))
    .group_by("job_name")
    .agg(pl.col("job_name").count().alias("count"))
    .sort("count", descending=True)
)

In [47]:
intern

job_name,count
str,u32
"""engineering intern""",56
"""mechanical engineering intern""",22
"""civil engineering intern""",19
"""software engineer intern""",19
"""software engineering intern""",18
…,…
"""data analyst – market & consum…",1
"""intern - software engineer in …",1
"""internship - process engineeri…",1


In [48]:
internship = (
    whole_df.filter(pl.col("job_name").str.contains("(?i)internship"))
    .group_by("job_name")
    .agg(pl.col("job_name").count().alias("count"))
    .sort("count", descending=True)
)

In [49]:
internship

job_name,count
str,u32
"""engineering internship""",11
"""electrical engineering interns…",5
"""internship for android develop…",5
"""mechanical engineering interns…",5
"""training / internship program …",4
…,…
"""front end developer (internshi…",1
"""mandatory internship - product…",1
"""internship: cyber security (50…",1


In [50]:
whole_df.select(pl.col("company_name").value_counts(sort=True)).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_28745,column_28746,column_28747,column_28748,column_28749,column_28750,column_28751,column_28752,column_28753,column_28754,column_28755,column_28756,column_28757,column_28758,column_28759,column_28760,column_28761,column_28762,column_28763,column_28764,column_28765,column_28766,column_28767,column_28768,column_28769,column_28770,column_28771,column_28772,column_28773,column_28774,column_28775,column_28776,column_28777,column_28778,column_28779,column_28780,column_28781
struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],…,struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2]
"{null,6266}","{""IBM"",2683}","{""Allied Universal"",1057}","{""CLBPTS"",668}","{""Bosch Group"",533}","{""Schneider Electric"",397}","{""260312-SOUTH FLORIDA REGION ADMIN"",380}","{""Novartis"",367}","{""Volvo Group"",342}","{""Lockheed Martin"",339}","{""Endeavor IT Solution"",305}","{""Open Systems Technologies"",280}","{""Explore Jobs Search"",266}","{""Weblee Technologies"",264}","{""The Boeing Company"",260}","{""Continental"",259}","{""Coders Brain Technology"",242}","{""Capgemini"",224}","{""IBM Careers"",220}","{""FullStack Labs"",215}","{""AECOM"",201}","{""241387-COMP & BEN ADMIN PROF FEES"",199}","{""Burlington Stores"",191}","{""Securitas US Business Unit"",176}","{""CACI-FEDERAL"",172}","{""Worley"",160}","{""Nagarro"",153}","{""Jobsbridge"",152}","{""Segula Technologies"",146}","{""Oowlish Technology"",143}","{""Publicis Groupe"",143}","{""Latitude"",136}","{""Sargent & Lundy"",135}","{""GardaWorld"",135}","{""Sonsoft"",134}","{""About Alstom"",129}","{""SAP"",127}",…,"{""Booksy"",1}","{""Who We Are : DELVE"",1}","{""APS Executive Search"",1}","{""IMG Advisory"",1}","{""Alfredo Salvatori"",1}","{""Quantum Space Systems"",1}","{""Nexi Group"",1}","{""Resi"",1}","{""SA Metal Group"",1}","{""Nova Consulting"",1}","{""Virtu Secure Web Services"",1}","{""Community Health Action of Staten"",1}","{""Sandfits Foundries"",1}","{""Health Heroes 1st"",1}","{""Focus Talent Group"",1}","{""Venn"",1}","{""Albany Med Health System"",1}","{""ingage"",1}","{""Standav"",1}","{""VEM SURPREENDER COM A GENTE"",1}","{""Delan"",1}","{""FOIT Group Pty"",1}","{""Wight & Company"",1}","{""Wipjobs"",1}","{""Coveris"",1}","{""Medical Information Technology"",1}","{""Fairfield Mountains Property Owners"",1}","{""PopBox Asia Services"",1}","{""Advize Group"",1}","{""Better Talent Advisors"",1}","{""AESC US"",1}","{""Weir Minerals Kazakhstan LLP"",1}","{""Ground Recruitment"",1}","{""ramblr.ai"",1}","{""UPL-"",1}","{""Mogo Finance Technology"",1}","{""Jump Trading"",1}"


In [51]:
whole_df.select(pl.col("seniority").value_counts(sort=True)).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14
struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2]
"{null,53579}","{""Senior IC"",17840}","{""Manager"",6688}","{""IC"",6623}","{""Unclear Seniority"",4593}","{""Staff IC"",3513}","{""Intern"",1982}","{""Junior IC"",1896}","{""Contract"",1106}","{""Director"",874}","{""Chief"",557}","{""Exec"",465}","{""Senior Manager"",273}","{""Senior Exec"",8}","{""Founder"",3}"


In [52]:
from polars_ds.diagnosis import DIA
import polars.selectors as cs

In [53]:
dia = DIA(whole_df)

In [54]:
dia.plot_null_distribution(cs.all())

GT(_tbl_data=shape: (18, 3)
┌──────────────────┬──────────────────────────────┬─────────┐
│ column           ┆ percentages in row groups    ┆ null%   │
│ ---              ┆ ---                          ┆ ---     │
│ str              ┆ struct[1]                    ┆ f64     │
╞══════════════════╪══════════════════════════════╪═════════╡
│ job_name         ┆ {[0.0, 0.0, … 0.0]}          ┆ 0.0     │
│ job_location     ┆ {[0.179, 0.1715, … 0.064]}   ┆ 0.11452 │
│ hours            ┆ {[0.3095, 0.3045, … 0.261]}  ┆ 0.27739 │
│ remote           ┆ {[0.7315, 0.7095, … 0.798]}  ┆ 0.76618 │
│ company_name     ┆ {[0.0565, 0.0535, … 0.0635]} ┆ 0.06266 │
│ …                ┆ …                            ┆ …       │
│ country          ┆ {[0.117, 0.1065, … 0.092]}   ┆ 0.09708 │
│ job_published_at ┆ {[0.1215, 0.1165, … 0.0945]} ┆ 0.10622 │
│ last_indexed     ┆ {[0.0, 0.0, … 0.0]}          ┆ 0.0     │
│ new              ┆ {[0.0, 0.0, … 0.0]}          ┆ 0.0     │
│ compensation     ┆ {[0.851, 0.865, … 0.8535]}   ┆ 0.85038 │
└──────────────────┴──────────────────────────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x717c902298d0>, _boxhead=Boxhead([ColInfo(var='column', type=<ColInfoTypeEnum.stub: 2>, column_label='column', column_align='left', column_width=None), ColInfo(var='percentages in row groups', type=<ColInfoTypeEnum.default: 1>, column_label='percentages in row groups', column_align='center', column_width=None), ColInfo(var='null%', type=<ColInfoTypeEnum.default: 1>, column_label='null%', column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname='job_name', group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname='job_location', group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname='hours', group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname='remote', group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname='company_name', group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname='education', group_label=None, built=False), RowInfo(rownum_i=6, group_id=None, rowname='tags_matched', group_label=None, built=False), RowInfo(rownum_i=7, group_id=None, rowname='tag_categories', group_label=None, built=False), RowInfo(rownum_i=8, group_id=None, rowname='categories', group_label=None, built=False), RowInfo(rownum_i=9, group_id=None, rowname='seniority', group_label=None, built=False), RowInfo(rownum_i=10, group_id=None, rowname='comp_est', group_label=None, built=False), RowInfo(rownum_i=11, group_id=None, rowname='language', group_label=None, built=False), RowInfo(rownum_i=12, group_id=None, rowname='city', group_label=None, built=False), RowInfo(rownum_i=13, group_id=None, rowname='country', group_label=None, built=False), RowInfo(rownum_i=14, group_id=None, rowname='job_published_at', group_label=None, built=False), RowInfo(rownum_i=15, group_id=None, rowname='last_indexed', group_label=None, built=False), RowInfo(rownum_i=16, group_id=None, rowname='new', group_label=None, built=False), RowInfo(rownum_i=17, group_id=None, rowname='compensation', group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title='Null Distribution', subtitle=None, preheader=None), _stubhead='column', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x717c90229bd0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x717c90229780>, <great_tables._gt_data.FormatInfo object at 0x717c90229b70>, <great_tables._gt_data.FormatInfo object at 0x717c90229e70>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'),

In [55]:
whole_df.select(pl.col("hours").value_counts(sort=True)).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15
struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2]
"{""Full-Time"",61589}","{null,27739}","{""Contract"",3727}","{""Part-Time"",2009}","{""Unclear"",1905}","{""Intern"",1048}","{""Temp"",701}","{""Hourly"",513}","{""Student"",280}","{""Trainee"",187}","{""Advisor"",94}","{""Gig"",83}","{""Commission"",82}","{""Grant"",27}","{""Conditional"",13}","{""Volunteer"",3}"


In [56]:
whole_df.select(pl.col("language").value_counts(sort=True)).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,column_37,column_38,column_39,column_40,column_41,column_42,column_43,column_44,column_45,column_46,column_47,column_48,column_49,column_50,column_51,column_52,column_53,column_54,column_55,column_56,column_57,column_58,column_59,column_60,column_61,column_62,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72
struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2]
"{""en"",75501}","{""en-us"",14252}","{""de"",1462}","{""en-gb"",1328}","{""fr"",1139}","{""es"",965}","{""pt-br"",836}","{null,584}","{""nl"",555}","{""ja"",375}","{""zh-cn"",362}","{""ko"",346}","{""pl"",312}","{""zh-tw"",275}","{""fr-ca"",252}","{""pt"",215}","{""sk"",214}","{""it"",158}","{""de-de"",131}","{""sv"",112}","{""ru-ru"",66}","{""fr-fr"",49}","{""id"",40}","{""no"",38}","{""tr"",38}","{""hu"",37}","{""en-za"",32}","{""ru"",30}","{""sl-si"",28}","{""en-au"",28}","{""cs"",23}","{""es-mx"",18}","{""uk"",16}","{""es-es"",15}","{""nl-nl"",12}","{""et"",11}","{""hr"",11}","{""fi"",11}","{""da"",11}","{""ro"",10}","{""it-it"",10}","{""tl"",9}","{""pl-pl"",7}","{""ca"",6}","{""ro-ro"",6}","{""cs-cz"",6}","{""vi"",5}","{""sv-se"",5}","{""lt-lt"",4}","{""af"",4}","{""el"",4}","{""cy"",4}","{""tr-tr"",3}","{""es-co"",3}","{""ka"",2}","{""sq"",2}","{""hu-hu"",2}","{""de-ch"",2}","{""sw"",2}","{""en-ca"",2}","{""zh"",2}","{""sl"",1}","{""ko-kr"",1}","{""he"",1}","{""lv"",1}","{""lt"",1}","{""gb"",1}","{""ar"",1}","{""sr"",1}","{""el-gr"",1}","{""th-th"",1}","{""es-ve"",1}","{""hy"",1}"


In [57]:
whole_df.select(pl.col("country").value_counts(sort=True)).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_146,column_147,column_148,column_149,column_150,column_151,column_152,column_153,column_154,column_155,column_156,column_157,column_158,column_159,column_160,column_161,column_162,column_163,column_164,column_165,column_166,column_167,column_168,column_169,column_170,column_171,column_172,column_173,column_174,column_175,column_176,column_177,column_178,column_179,column_180,column_181,column_182
struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],…,struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2],struct[2]
"{""United States"",44581}","{""India"",10276}","{null,9708}","{""United Kingdom"",4191}","{""Germany"",3560}","{""Canada"",2362}","{""Brazil"",1783}","{""France"",1530}","{""Australia"",1340}","{""Mexico"",1015}","{""China"",948}","{""Poland"",927}","{""Singapore"",902}","{""Spain"",857}","{""Netherlands"",854}","{""South Africa"",774}","{""Israel"",769}","{""Philippines"",666}","{""Italy"",587}","{""Romania"",574}","{""Malaysia"",564}","{""Japan"",509}","{""Ireland"",496}","{""Belgium"",480}","{""Sweden"",470}","{""Switzerland"",411}","{""Portugal"",401}","{""Colombia"",383}","{""Argentina"",371}","{""Austria"",334}","{""Thailand"",325}","{""Saudi Arabia"",309}","{""United Arab Emirates"",303}","{""Czech Republic"",294}","{""Taiwan"",281}","{""Egypt"",275}","{""Hungary"",263}",…,"{""Zambia"",2}","{""Sierra Leone"",2}","{""Kyrgyzstan"",2}","{""Democratic Republic Of The Congo"",2}","{""Antarctica"",2}","{""Zimbabwe"",2}","{""Ethiopia"",2}","{""Fiji"",2}","{""Trinidad And Tobago"",2}","{""Belize"",2}","{""Greenland"",1}","{""Wallis And Futuna"",1}","{""Mozambique"",1}","{""Tajikistan"",1}","{""Sudan"",1}","{""Afghanistan"",1}","{""Liberia"",1}","{""Aruba"",1}","{""Central African Republic"",1}","{""Yemen"",1}","{""San Marino"",1}","{""Turkmenistan"",1}","{""Marshall Islands"",1}","{""Djibouti"",1}","{""Saint Lucia"",1}","{""Togo"",1}","{""Benin"",1}","{""Laos"",1}","{""Bermuda"",1}","{""Faroe Islands"",1}","{""Vanuatu"",1}","{""Brunei"",1}","{""Guinea"",1}","{""Guyana"",1}","{""Gabon"",1}","{""Saint Kitts And Nevis"",1}","{""Libya"",1}"


In [58]:
date_data_new = whole_df.select(cs.date())
bool_data_new = whole_df.select(cs.by_dtype(pl.Boolean))
string_data_new = whole_df.select(cs.string(include_categorical=True))
nested_data_new = whole_df.select(
    cs.by_name("tags_matched", "tag_categories", "categories")
)
num_data_new = whole_df.select(cs.float())

In [59]:
whole_df.select(pl.col("job_location").value_counts(sort=True))

job_location
struct[2]
"{null,11452}"
"{""Remote"",1316}"
"{""United States"",1284}"
"{""Bangalore, India"",942}"
"{""New York, NY"",453}"
…
"{""US - Wisconsin - Madison"",1}"
"{""Karachi, Sindh, 74000 Pakistan"",1}"
"{""800 E. Middlefield Rd"",1}"


In [60]:
print(f"date type columns:{date_data_new.columns}")
print(f"bool type columns:{bool_data_new.columns}")
print(f"string type columns:{string_data_new.columns}")
print(f"nested type columns:{nested_data_new.columns}")

date type columns:['job_published_at', 'last_indexed']
bool type columns:['remote']
string type columns:['job_name', 'job_location', 'hours', 'company_name', 'education', 'seniority', 'comp_est', 'language', 'city', 'country', 'new']
nested type columns:['tags_matched', 'tag_categories', 'categories']


In [61]:
missing = (
    whole_df.select(pl.all().is_null().sum())
    .melt(value_name="missing")
    .filter(pl.col("missing") > 0)
)